In [ ]:
import pandas as pd
df_AM_FM_THD4 = pd.read_excel("Wahania_Modulacja_Nosna_CCOS.xlsx", sheet_name="AM_FM_ccos_THD_4%")
df_AM_FM_THD4.drop(columns=df_AM_FM_THD4.filter(regex='Unnamed').columns, inplace=True)

In [ ]:
df_AM_FM_THD4

,Uc [V],230,Lp.,fm [Hz],DUU_1 [%],PST/1,DUU_3 [%],PST/3,DUU_0333 [%],"PST/0,333"
0,fc [Hz],50,1,0.033333,1.780011,NaN,5.280021,NaN,0.585541,NaN
1,Pst [-],const,2,0.066667,1.400011,NaN,4.146021,NaN,0.480011,NaN
2,Iph [deg],0,3,0.100000,1.201071,NaN,3.604521,NaN,0.410011,NaN
3,NaN,RECTANGLE,4,0.133333,1.110011,NaN,3.300021,NaN,0.365521,NaN
4,NaN,NaN,5,0.166667,1.031971,NaN,3.100011,NaN,0.350011,NaN
...,...,...,...,...,...,...,...,...,...,...
269,NaN,NaN,270,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,NaN,NaN,271,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,NaN,NaN,272,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,NaN,NaN,273,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import pandas as pd
from scipy.signal import hilbert, square, sawtooth
from matplotlib import pyplot as plt, animation
import numpy as np
import os
from scipy.fft import fft

fs =20480.0  # czestotliwosc probkowania
T =600.0  # czas trwania
dt = 1 / fs  # krok czasowy
n = T * fs  # liczba probek sygnału
t = np.arange(0,T+dt,dt)  # wektor czasu
A = 230 * np.sqrt(2)
f=50.0

def sygnal(A,f):
    mc_4 = 0.92
    u = np.sin(2*np.pi*50*t);
    u = np.clip(u, -mc_4, mc_4)
    rms = np.sqrt(np.mean(u ** 2))
    u = (u / rms) * 230
    Y = fft(u)
    L=len(u)
    P2 = abs(Y/L)
    P1 = P2[:L//2+1]
    P1[1:-1] = 2*P1[1:-1]
    tol = 1e-6
    Y[abs(Y) < tol] = 0
    theta = np.angle(Y)
    index = np.where(P1 == np.max(P1))[0]
    u1 = np.zeros((len(u),))
    for h in range(1,40,2):
        u1 += (1+DUU/2/100*square(2*np.pi*fm*t))*P1[(h*index)[0]]*np.sin(2*np.pi*h*50*t+theta[(h*index)[0]]+np.pi*DUU/2/100*(sawtooth(2*np.pi*fm*t,0.5)+1)*0.5)
    return u1

file_path='/content/drive/MyDrive/wyniki_modulacja_AM_FM_THD4.csv'

if os.path.exists(file_path):
    df=pd.read_csv(file_path)

else:
    df=pd.DataFrame([])
nazwy_kolumn = ['AM_FM_THD4_3_A','AM_FM_THD4_3_f']
df2 = pd.DataFrame([], columns=nazwy_kolumn)
for n, (i,j) in enumerate(zip(df_AM_FM_THD4["fm [Hz]"][0:201], df_AM_FM_THD4["DUU_3 [%]"][0:201])):
  DUU=j
  fm=i

  obwiednia = hilbert(sygnal(A,f))

  amplituda = np.abs(obwiednia)
  faza = np.angle(obwiednia)
  czestotliwosc = (np.diff(faza) / (2.0*np.pi) * fs)

  b_w_A = ((abs(((1 + DUU / 200 * square(2 * np.pi * fm * t)) * A) - amplituda)) / ((1 + DUU / 200 * square(2 * np.pi * fm * t)) * A)) * 100
  b_w_A=b_w_A[::205]
  t1=t[1:]
  b_w_f= ((abs((50 + DUU / 200 * square(2 * np.pi * fm * t1)) - czestotliwosc)) / (50 + DUU / 200 * square(2 * np.pi * fm * t1))) * 100
  b_w_f=b_w_f[::205]

  df3 = pd.DataFrame(zip(b_w_A, b_w_f), columns=nazwy_kolumn)
  df2 = pd.concat([df2, df3], ignore_index=True)

if df.empty:
  df=df2
else:
  df=df.merge(df2, left_index=True, right_index=True)

df.to_csv(file_path,index=False)



NameError: ignored

In [ ]:
t



array([0.0000000e+00, 5.0000000e-05, 1.0000000e-04, ..., 5.9999990e+02,
       5.9999995e+02, 6.0000000e+02])